In [11]:
class InfiniteTabs:
    def __init__(
        self,
        children=[],
        titles=[],
        factory=None,
        factory_text='Add tab',
        initial_factory_tab=False,
        max_tabs=None, 
        *args, **kwargs
    ):
        self._last_uid = 0
        self._widget = widgets.Tab(children=[], *args, **kwargs)
        self._tabs = []
        self._has_factory = (factory is not None)
        if self._has_factory:
            self._factory = factory
            self._max_tabs = max_tabs
            self._add_text = factory_text
            add_button = widgets.Button(description=self._add_text, button_style='info')
            add_button.on_click(lambda _: self._add_tab_from_factory())
            add_tab = self._add_tab(content=add_button, title=self._add_text)
            self.on_tab_select(add_tab, lambda _: self._add_tab_from_factory())
        for title, child in zip(titles, children):
            self.add_tab(content=child, title=title)
        if self._has_factory and initial_factory_tab:
            self._add_tab_from_factory()

    def display(self):
        display(self._widget)

    def close(self):
        self._widget.close()

    def set_title(self, uid, title):
        index = self._tab_index(uid)
        if index is not None:
            self._tabs[index]['title'] = title
            self._widget.set_title(index, title)

    @property
    def tabs_count(self) -> int:
        if self._has_factory:
            return max(0, len(self._tabs) - 1)
        else:
            return len(self._tabs)

    def _add_tab_from_factory(self):
        return self.add_tab(*self._factory(self._last_uid))

    def _add_tab(self, content=None, title=None, index=None):
        if self._max_tabs is not None and self.tabs_count >= self._max_tabs:
            return
        content = content or widgets.HBox()
        tab = { 'uid': self._last_uid, 'title': title, 'content': content }
        index = len(self._tabs) if index is None else index
        self._tabs.insert(index, tab)
        self._last_uid += 1
        self._update()
        return tab['uid']

    def add_tab(self, content=None, title=None):
        """
        This is a public alias to _add_tab, which inserts the new tab before the last one,
        which is reserved by the "Add button tab" if there is one
        """
        index = len(self._tabs) - 1 if self._has_factory else len(self._tabs)
        return self._add_tab(content, title, index)

    def remove_tab(self, uid):
        index = self._tab_index(uid)
        if index is None:
            raise Exception(f'no tab matching uid {uid}')
        self._tabs.pop(index)
        self._update()

    def on_change(self, callback):
        self._widget.observe(callback, 'selected_index')

    def on_tab_select(self, uid, callback):
        self.on_change(lambda e: callback(e) if e.new == self._tab_index(uid) else False)

    def _tab_index(self, uid):
        try:
            return next(index for index, tab in enumerate(self._tabs) if tab['uid'] == uid)
        except:
            return None

    def _update(self):
        self._widget.children = tuple([ tab['content'] for tab in self._tabs ])
        for index, tab in enumerate(self._tabs):
            if tab['title'] is not None:
                self._widget.set_title(index, tab['title'])